In [30]:
import numpy as np
from astropy.io import fits
import time


def running_stats(filenames):
    '''Calculates the running mean and stdev for a list of FITS files using Welford's method.'''
    n = 0
    for filename in filenames:
        hdulist = fits.open(filename)
        data = hdulist[0].data
        if n == 0:
            mean = np.zeros_like(data)
            s = np.zeros_like(data)

        n += 1
        delta = data - mean
        mean += delta/n
        s += delta*(data - mean)
        hdulist.close()

    s /= n - 1
    np.sqrt(s, s)

    if n < 2:
        return mean, None
    else:
        return mean, s



In [36]:
def median_bins_fits(filenames, B):
    mean, std = running_stats(filenames)
    mean = mean.reshape(200*200,1)
    std = std.reshape(200*200,1)
    minval,maxval = mean-std,mean+std
    
    lesser_count = np.zeros((200*200,1))
    hist = np.zeros((200*200,B))
    for index,file in enumerate(filenames):
        data = fits.open(file)[0].data
        data = data.reshape(200*200,1)
        for pixel in data:   
                bins = np.linspace(minval[index],maxval[index],B)
                if data[index] < minval[index]:
                    lesser_count[index] += 1
                    
                elif minval[index] <= data[index] <= maxval[index]:
                    hist[index,np.digitize(data[row,col],bins)-1] += 1   
                    
    mean = mean.reshape((200,200))
    std = std.reshape((200,200))
    lesser_count = lesser_count.reshape((200,200))
    hist = hist..reshape((200,200,B))
    return (mean,std,lesser_count,hist)
    
def median_approx_fits(filenames, B):
    N = len(filenames)
    mean, std, total, hist = median_bins_fits(filenames, B)
    median = np.empty((200,200))
    # compute the median for each pixel
    for row in range(200):
        for col in range(200):
            for index,count in enumerate(hist[row,col]):
                total[row,col] += count
                if total[row,col] >= (N+1)/2: 
                    median[row,col] = mean[row,col] + std[row,col]*( (2*index + 1)/B -1)
                    break
    return median
    


SyntaxError: invalid syntax (<ipython-input-36-4ed4ecd3093e>, line 23)

In [37]:
%%timeit

# You can use this to test your function.
# Any code inside this `if` statement will be ignored by the automarker.
if __name__ == '__main__':
  # Run your function with examples from the question.
    mean, std, left_bin, bins = median_bins_fits(['image0.fits', 'image1.fits', 'image2.fits'], 5)
    median = median_approx_fits(['image0.fits', 'image1.fits', 'image2.fits'], 5)
    print(mean[100,100])
    print(std[100,100])
    print(left_bin[100, 100])
    print(bins[100, 100, :])
    print(median[100, 100])

IndexError: index 5 is out of bounds for axis 2 with size 5

In [22]:
a = np.random.rand(20,20)
a = a.reshape(a.size,1)
a.shape


(400, 1)